In [2]:
!pip install openai tqdm pandas

In [11]:
import pandas as pd
import json
import time
from tqdm import tqdm
from openai import OpenAI
from google.colab import userdata

api_key = userdata.get('UPSTAGE_API_KEY')
client = OpenAI(
    api_key=api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

MODEL_NAME = "solar-pro"
BATCH_SIZE = 50

try:
    df = pd.read_csv('data2.csv')
    if df['title'].str.startswith('http').any():
        print("⚠️ 경고: 'title' 컬럼에 유튜브 주소가 포함된 것 같습니다. 이 코드는 '한글 제목'을 분석합니다.")
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame()

def extract_trend_keywords(titles_batch):
    system_prompt = """
    당신은 '유튜브 바이럴 트렌드 분석가'입니다.
    제공된 제목에서 마케팅에 활용할 수 있는 **'핵심 트렌드 키워드'**를 추출하세요.

    [추출 원칙]
    1. **구체적인 아이템/신조어 우선**: '쿠키'보다는 '두바이 쫀득쿠키', '아이스크림'보다는 '요아정'과 같이 고유명사나 유행하는 복합명사를 그대로 추출하세요.
    2. **불필요한 단어 제거**: '브이로그', '추천', '영상', '리뷰', 'ㅋㅋㅋ', '존맛' 같은 일반적인 수식어나 감탄사는 제외하세요.
    3. **맥락 파악**: 제목이 특정 챌린지나 밈(Meme)을 다룬다면 해당 밈의 이름을 정확히 추출하세요 (예: '꽁꽁 얼어붙은 한강').

    [출력 형식]
    반드시 JSON 형식으로 반환하세요:
    {
        "results": [
            {"title": "원본 제목", "keywords": ["키워드1", "키워드2", "키워드3"]}
        ]
    }
    """

    user_prompt = f"""
    아래 제목 리스트를 분석하여 JSON으로 출력해줘.

    [제목 리스트]
    {json.dumps(titles_batch, ensure_ascii=False)}
    """

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content).get('results', [])
    except Exception as e:
        print(f"API Error: {e}")
        return []

if not df.empty:
    print("Upstage Solar를 통한 트렌드 키워드 추출 시작...")

    all_keywords = []
    titles = df['title'].tolist()

    for i in tqdm(range(0, len(titles), BATCH_SIZE)):
        batch_titles = titles[i : i + BATCH_SIZE]

        results = extract_trend_keywords(batch_titles)

        for t in batch_titles:
            found = next((item for item in results if item.get('title') == t), None)
            if found:
                k_str = ", ".join(found.get('keywords', []))
                all_keywords.append(k_str)
            else:
                all_keywords.append("")

    df['trend_keywords'] = all_keywords

    output_file = 'data_with_trend_keywords.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n분석 완료! '{output_file}' 저장됨.")

    print("\n[추출 결과 미리보기]")
    print(df[['title', 'trend_keywords']].head())

Upstage Solar를 통한 트렌드 키워드 추출 시작...


100%|██████████| 16/16 [04:42<00:00, 17.66s/it]


분석 완료! 'data_with_trend_keywords.csv' 저장됨.

[추출 결과 미리보기]
                                        title trend_keywords
0   요즘 핫한 10분 끓인 너구리를 친구놈한테 먹여봤습니다 #제품리뷰 #너구리            너구리
1  &quot;자매품 아닌데&quot; 빵 대표의 설움 #shorts / YTN               
2                       CG아님. #상남자 #전용 #아이스크림          아이스크림
3         모든 백수저와 흑수저를 입다물게 만든 도전자 #흑백요리사 시즌2      흑백요리사 시즌2
4                ‘흑백요리사2’ 드디어 이해된 손종원이 차갑던 이유         흑백요리사2


In [12]:
all_keywords

['너구리',
 '',
 '아이스크림',
 '흑백요리사 시즌2',
 '흑백요리사2',
 '두바이 쫀득쿠키',
 '두바이 쫀득 쿠키',
 '',
 '',
 '흑백요리사 시즌2',
 '침착맨 버거',
 '치즈케이크',
 '치즈 케이크',
 '',
 '두바이쫀득쿠키',
 '대창 닭발',
 '두쫀쿠',
 '흑백요리사2, 쌤쌤쌤',
 '흑백요리사2',
 '동결건조',
 '요거트치즈케이크',
 '',
 '두바이 쫀득김밥',
 '',
 '',
 '흑백요리사2, 흑수저',
 '치즈케이크',
 '',
 '',
 '',
 '치즈케이크, 빠다코코넛',
 '',
 '두바이쫀득쿠키, 두바이쫀득김밥',
 '두바이',
 '',
 '',
 '두바이 쫀득 쿠키',
 '',
 '',
 '',
 '',
 '두바이쫀득쿠키',
 '',
 '흑백요리사 시즌2',
 '',
 '',
 '키보드 키링',
 '두쫀쿠',
 '신비한 과자가게 연남',
 '',
 '편의점 꾸덕젤리',
 '일본 치즈케이크',
 '12월 편의점 신박한 제품',
 '대용량 간식템',
 '용사라면',
 '2025 밈으로 만든 노래',
 '불닭미역탕면 다이어트 버전',
 '스팀에어프라이어',
 '두바이쫀득쿠키',
 '일본 스퀴시',
 '파리바게트 베리밤',
 '성심당 딸기가득 말차시루',
 '',
 '크리스마스 초간단 간식',
 '상하이 편의점 신기한 아이스크림',
 '2025년 유행 음식 모음',
 '까르보불닭볶음면 페스츄리소세지빵',
 '속초 88 생선구이',
 '두바이 쫀득쿠키, 키딩 챌린지',
 '엄마표 모둠전',
 '파리바게트 베리밤',
 '이마트24 이 주의 신상',
 '피자스쿨 오지치즈포테이토피자',
 '두쫀쿠',
 '모란시장 떡집',
 '흑백요리사 시즌2',
 '중식 폭주족 삼겹살 피라미드, 흑백요리사 시즌2',
 '2025년 유행 음식들',
 '오이라면',
 '',
 '12월 트레이더스 신상',
 '미국 유행 피자먹는 법',
 '2025 회식 핫플, 회식 맛집',
 '두찜 신메뉴',
 '디저트 맛집 TOP6',
 '',

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import matplotlib.font_manager as fm
import os

keywords_series = df['trend_keywords'].dropna().astype(str)
keywords_list = keywords_series.str.split(',')

all_tokens = [
    token.replace(' ', '')
    for sublist in keywords_list
    for token in sublist
    if token.replace(' ', '')
]

keyword_counts = Counter(all_tokens)

top_20 = keyword_counts.most_common(20)
print(f"총 추출된 고유 키워드 개수 (공백 제거 후): {len(keyword_counts)}")
print("\n[상위 20개 트렌드 키워드 - 공백 제거됨]")
for word, count in top_20:
    print(f"{word}: {count}회")


총 추출된 고유 키워드 개수 (공백 제거 후): 755

[상위 20개 트렌드 키워드 - 공백 제거됨]
두바이쫀득쿠키: 71회
두쫀쿠: 40회
불닭미역탕면: 13회
치즈케이크: 11회
두바이: 9회
흑백요리사시즌2: 8회
두바이초콜릿: 7회
흑백요리사2: 5회
빠다코코넛: 5회
쫀득쿠키: 5회
신라면골드: 5회
요거트치즈케이크: 4회
일본치즈케이크: 4회
단어리듬게임: 4회
두바이디저트: 4회
두바이초콜렛: 4회
두바이수건케이크: 4회
딸기생크림빵: 3회
불닭볶음탕면: 3회
세븐일레븐: 3회
